In [1]:
import importlib
import data
from gurobipy import *

# get data from excel
dataframes = data.get_all_pandas()

LOADING DATA: 7it [00:31,  3.12s/it]                       

In [3]:
import variables
import contraintes
import importlib
importlib.reload(variables) #to remive in final .py
importlib.reload(contraintes) #to remive in final .py

<module 'contraintes' from 'c:\\Users\\remgo\\Desktop\\ProjetST7\\git\\contraintes.py'>

In [5]:
from openpyxl import Workbook
from datetime import datetime

def show_solution(model):
    # print(var_dict)
    print("Objectif minimal: ", m.objVal)
    #build excel of tasks
    workbook = Workbook()
    sheet = workbook.active
    sillons_df = dataframes["sillons_df"]

    sheet["A1"] = "Id_tache"
    sheet["B1"] = "tache"
    sheet["C1"] = "minute"
    sheet["D1"] = "HARR"
    
    sheet["E1"] = "JDEP"
    row = 2
    for tache in var_dict.keys():
        for sillon_id in var_dict[tache].keys():
            sheet[f"B{row}"] = tache
            sheet[f"C{row}"] = var_dict[tache][sillon_id].X
            sheet[f"A{row}"] = sillon_id
            sheet[f"D{row}"] = sillons_df[sillons_df["train_id"]==sillon_id]["HARR"].iloc[0]
            sheet[f"E{row}"] = sillons_df[sillons_df["train_id"]==sillon_id]["JDEP"].iloc[0]
            row += 1

    currentDateAndTime = datetime.now().strftime("%d-%m_%H-%M-%S")
    workbook.save(filename=f"results/{data.NAME}-{currentDateAndTime}.xlsx")
# show_solution(m)


In [6]:
# create model
m = Model()

# generate variables (integers)
var_dict = variables.generate_variables(m, dataframes)
# generate variables
B, OCCUPATIONS = contraintes.generate_contraintes(m, dataframes, var_dict)

final_train_task = dataframes["sillons_df"]
final_train_task = final_train_task[final_train_task["JARR"]==final_train_task["JARR"].max()]
final_train_task = final_train_task[final_train_task["HDEP"]==final_train_task["HDEP"].max()]["train_id"].iloc[0]
final_train_task = var_dict["essai de frein départ"][final_train_task]
print(final_train_task)
m.setObjective(final_train_task, GRB.MINIMIZE)

m.update()
m.printStats()
# m.write("genconstr.mps")
# m.write("test.lp")
m.optimize()
# m.write("test.lp")

show_solution(m)

Initialising variables: 100%|██████████| 217/217 [00:00<00:00, 911.77it/s]


debug sillons
5408609/08/202203:5023:59:0014/08/2022
[2347]
43124608/08/202200:00:0011:0408/08/2022
debug sillons
5408611/08/202203:5023:59:0014/08/2022
[159]
45023708/08/202200:00:0011:4710/08/2022
debug sillons
5408612/08/202203:5023:59:0014/08/2022
[160, 225, 323, 687, 1063, 1106, 1497, 1703, 2349]
45023708/08/202200:00:0011:4711/08/2022
5673008/08/202200:00:0002:3211/08/2022
5673008/08/202200:00:0002:3211/08/2022
4126408/08/202200:00:0022:3310/08/2022
5423408/08/202200:00:0022:0110/08/2022
4126408/08/202200:00:0022:3310/08/2022
4126408/08/202200:00:0022:3310/08/2022
43164008/08/202200:00:0011:0911/08/2022
43124608/08/202200:00:0011:0411/08/2022
debug sillons
5408613/08/202203:5023:59:0014/08/2022
[1107]
4125408/08/202200:00:0023:1711/08/2022
debug sillons
5408614/08/202203:5023:59:0014/08/2022
[162, 1061]
41202308/08/202200:00:0002:0713/08/2022
5423408/08/202200:00:0022:0112/08/2022
debug sillons
5408615/08/202203:5023:59:0015/08/2022
[102, 227, 326, 624, 690, 1109, 1399, 1458, 150

Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

In [ ]:
##### Respect du nombre de voies de chantier #####
#debug
# print(B)



taches_df = dataframes["taches_df"]
chantiers_df = dataframes["chantiers_df"]
chantier_cycles = {}
for chantier in set(taches_df["Chantier"].values):
    chantier_cycles[chantier, "end"] = taches_df[taches_df["Chantier"] == chantier][taches_df.Ordre ==
                                                                                        taches_df[taches_df["Chantier"] == chantier].Ordre.max()]["Type de tache humaine"].iloc[0]
    chantier_cycles[chantier, "start"] = taches_df[taches_df["Chantier"] == chantier][taches_df.Ordre ==
                                                                                    taches_df[taches_df["Chantier"] == chantier].Ordre.min()]["Type de tache humaine"].iloc[0]
print(chantier_cycles)
def get_all_tasks_by_name(name):
    # print("get_all : ", name)
    return var_dict[name].keys()
def b(tache_i, tache_j, tache_name, duree=0):
    if(tache_name,tache_j,tache_i) in B.keys():
        # print("HEY === ",(tache_name,tache_j,tache_i))
        bb = B[tache_name,tache_j,tache_i]
        # print(bb.X, end=",")
        return bb.X
    elif (tache_name,tache_i,tache_j) in B.keys():
        # print("HEY ELIF === ",(tache_name,tache_i,tache_j))
        bb = B[tache_name,tache_i,tache_j]
        # print(1-bb.X, end=",")
        return 1-bb.X
            
        
    else:
        print("ERROR === ", (tache_name,tache_i,tache_j))
               

OCCUPATIONS = []
def add_occupation_constr(chantier, tache_debut):
    
    occupation = 0 
    occupation = sum([b(tache_chantier,
                                tache_debut,
                                chantier_cycles[chantier, "start"])
                            for tache_chantier in get_all_tasks_by_name(chantier_cycles[chantier, "start"])])
    duree = taches_df[taches_df["Type de tache humaine"]
                        == chantier_cycles[chantier, "end"]]["Durée"].iloc[0]
    # print(occupation)
    # print("-----------")
    
    temp = sum([b(tache_chantier,
                                            tache_debut,
                                            chantier_cycles[chantier, "end"],
                                            duree= duree)
                                        for tache_chantier in get_all_tasks_by_name(chantier_cycles[chantier, "end"])])
    # print(temp)
    occupation = occupation -temp
    if occupation!= 0 :print("NON ZERO",occupation)
    OCCUPATIONS.append(occupation)
# for chantier in set(chantiers_df["Chantier"].values):
for chantier in ["WPY_FOR"]:
    print(f"Adding occupation constrainst for: {chantier}...")
    print("start task: ",chantier_cycles[chantier, "start"])
    print("end task: ",chantier_cycles[chantier, "end"])
    for tache_debut in get_all_tasks_by_name(chantier_cycles[chantier, "start"]):
        add_occupation_constr(chantier, tache_debut)
print(OCCUPATIONS)
# print(B[("arrivée Reception","43124608/08/202200:00:0011:0408/08/2022","45023708/08/202200:00:0011:4708/08/2022")])

In [4]:
# tests 
importlib.reload(contraintes)
importlib.reload(variables)
# create model
m = Model()

# generate variables (integers)
var_dict = variables.generate_variables(m, dataframes)
# generate variables
contraintes.generate_contraintes(m, dataframes, var_dict)

Set parameter Username
Academic license - for non-commercial use only - expires 2024-01-26


Initialising variables: 100%|██████████| 217/217 [00:00<00:00, 893.01it/s]
c:\Users\remgo\Desktop\ProjetST7\git\contraintes.py:175: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  tache_precedante = taches_df[taches_df["Ordre"] == ordre -
c:\Users\remgo\Desktop\ProjetST7\git\contraintes.py:183: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  list_wagons = list(correspondances_df[correspondances_df["train_id"] ==
c:\Users\remgo\Desktop\ProjetST7\git\contraintes.py:190: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  sillon_arr = correspondances_df[correspondances_df["id_wagon"] ==


debug sillons
5408609/08/202203:5023:59:0014/08/2022
[2347]
43124608/08/202200:00:0011:0408/08/2022
debug sillons
5408611/08/202203:5023:59:0014/08/2022
[159]
45023708/08/202200:00:0011:4710/08/2022
debug sillons
5408612/08/202203:5023:59:0014/08/2022
[160, 225, 323, 687, 1063, 1106, 1497, 1703, 2349]
45023708/08/202200:00:0011:4711/08/2022
5673008/08/202200:00:0002:3211/08/2022
5673008/08/202200:00:0002:3211/08/2022
4126408/08/202200:00:0022:3310/08/2022
5423408/08/202200:00:0022:0110/08/2022
4126408/08/202200:00:0022:3310/08/2022
4126408/08/202200:00:0022:3310/08/2022
43164008/08/202200:00:0011:0911/08/2022
43124608/08/202200:00:0011:0411/08/2022
debug sillons
5408613/08/202203:5023:59:0014/08/2022
[1107]
4125408/08/202200:00:0023:1711/08/2022
debug sillons
5408614/08/202203:5023:59:0014/08/2022
[162, 1061]
41202308/08/202200:00:0002:0713/08/2022
5423408/08/202200:00:0022:0112/08/2022
debug sillons
5408615/08/202203:5023:59:0015/08/2022
[102, 227, 326, 624, 690, 1109, 1399, 1458, 150

c:\Users\remgo\Desktop\ProjetST7\git\contraintes.py:263: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  chantier_cycles[chantier, "end"] = taches_df[taches_df["Chantier"] == chantier][taches_df.Ordre ==
c:\Users\remgo\Desktop\ProjetST7\git\contraintes.py:265: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  chantier_cycles[chantier, "start"] = taches_df[taches_df["Chantier"] == chantier][taches_df.Ordre ==


Adding occupation constrainst for: WPY_REC...


({('essai de frein départ',
   '40000609/08/202215:4623:59:0014/08/2022',
   '40000609/08/202215:4623:59:0014/08/2022'): <gurobi.Var *Awaiting Model Update*>,
  ('essai de frein départ',
   '40000609/08/202215:4623:59:0014/08/2022',
   '40001609/08/202222:4623:59:0014/08/2022'): <gurobi.Var *Awaiting Model Update*>,
  ('essai de frein départ',
   '40000609/08/202215:4623:59:0014/08/2022',
   '40011009/08/202220:5723:59:0014/08/2022'): <gurobi.Var *Awaiting Model Update*>,
  ('essai de frein départ',
   '40000609/08/202215:4623:59:0014/08/2022',
   '43103509/08/202215:0223:59:0014/08/2022'): <gurobi.Var *Awaiting Model Update*>,
  ('essai de frein départ',
   '40000609/08/202215:4623:59:0014/08/2022',
   '43109109/08/202206:1923:59:0014/08/2022'): <gurobi.Var *Awaiting Model Update*>,
  ('essai de frein départ',
   '40000609/08/202215:4623:59:0014/08/2022',
   '4420309/08/202217:5523:59:0014/08/2022'): <gurobi.Var *Awaiting Model Update*>,
  ('essai de frein départ',
   '40000609/08/202